In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pandas.io.json import json_normalize
import numpy as np
from geopy.distance import geodesic

``Importo collections sobre los requisitos en San Francisco``

In [2]:
dbName = "Crunchbase_Dataset"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL)
db = client.get_database()

In [3]:
curse = db.gamers_over_1.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
gamer_companies = list(curse)
df= pd.DataFrame(gamer_companies)
df.head()

,_id,name,latitude,longitude,location
0,5ef9f15345de338078b9ac2e,Ustream,37.392936,-122.079480,"{'type': 'Point', 'coordinates': [-122.07948, ..."
1,5ef9f15345de338078b9ac2f,Revision3,37.757758,-122.388243,"{'type': 'Point', 'coordinates': [-122.388243,..."
2,5ef9f15345de338078b9ac30,CastTV,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,..."
3,5ef9f15345de338078b9ac31,hi5,37.788668,-122.400558,"{'type': 'Point', 'coordinates': [-122.400558,..."
4,5ef9f15345de338078b9ac32,Metacafe,37.437328,-122.159928,"{'type': 'Point', 'coordinates': [-122.159928,..."


In [4]:
curse = db.vegan.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
vegan = list(curse)
df_vegan= pd.DataFrame(vegan)
df_vegan.head()

,_id,name,latitude,longitude,location
0,5ef9f16e45de338078b9ac4c,Ananda Fuara,37.777693,-122.416353,"{'type': 'Point', 'coordinates': [-122.4163533..."
1,5ef9f16e45de338078b9ac4d,Golden Era Vegan,37.781495,-122.416822,"{'type': 'Point', 'coordinates': [-122.4168223..."
2,5ef9f16e45de338078b9ac4e,Thai Idea Vegetarian Restaurant,37.783395,-122.419141,"{'type': 'Point', 'coordinates': [-122.419141,..."
3,5ef9f16e45de338078b9ac4f,Brenda's French Soul Food,37.782896,-122.418897,"{'type': 'Point', 'coordinates': [-122.4188974..."
4,5ef9f16e45de338078b9ac50,Ike's Sandwiches,37.785146,-122.419629,"{'type': 'Point', 'coordinates': [-122.4196290..."


In [5]:
curse = db.airport.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
airports = list(curse)
df_airports= pd.DataFrame(airports)
df_airports.head()

,_id,name,latitude,longitude,location
0,5ef9f12245de338078b9ac0a,West Sunset RC Flying Field,37.748486,-122.497641,"{'type': 'Point', 'coordinates': [-122.4976407..."
1,5ef9f12245de338078b9ac0b,Design To Win Foundation,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994..."
2,5ef9f12245de338078b9ac0c,SFO,37.785729,-122.410446,"{'type': 'Point', 'coordinates': [-122.4104463..."
3,5ef9f12245de338078b9ac0d,MUrgency Inc. (Airport Assistance; Tech Innova...,37.766902,-122.419558,"{'type': 'Point', 'coordinates': [-122.419558,..."
4,5ef9f12245de338078b9ac0e,Airport Express,37.778891,-122.402358,"{'type': 'Point', 'coordinates': [-122.402358,..."


In [6]:
curse = db.clubs.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
clubs = list(curse)
df_clubs= pd.DataFrame(clubs)
df_clubs.head()

,_id,name,latitude,longitude,location
0,5ef9f13e45de338078b9ac1a,Zero Zero,37.781636,-122.402083,"{'type': 'Point', 'coordinates': [-122.4196054..."
1,5ef9f13e45de338078b9ac1b,Smuggler's Cove,37.779403,-122.423372,"{'type': 'Point', 'coordinates': [-122.4196054..."
2,5ef9f13e45de338078b9ac1c,The Valencia Room,37.762506,-122.421412,"{'type': 'Point', 'coordinates': [-122.4196054..."
3,5ef9f13e45de338078b9ac1d,The Grand,37.779100,-122.397900,"{'type': 'Point', 'coordinates': [-122.4196054..."
4,5ef9f13e45de338078b9ac1e,Edinburgh Castle Pub,37.786127,-122.418957,"{'type': 'Point', 'coordinates': [-122.4196054..."


In [7]:
curse = db.school.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
schools = list(curse)
df_schools= pd.DataFrame(schools)
df_schools.head()

,_id,name,latitude,longitude,location
0,5ef9f16145de338078b9ac43,John Swett Alternative Elementary School,37.780412,-122.422916,"{'type': 'Point', 'coordinates': [-122.422916,..."
1,5ef9f16145de338078b9ac44,Patricia's Green,37.776369,-122.424479,"{'type': 'Point', 'coordinates': [-122.4244790..."
2,5ef9f16145de338078b9ac45,Chinese American International School,37.775275,-122.422111,"{'type': 'Point', 'coordinates': [-122.4221108..."
3,5ef9f16145de338078b9ac46,C5 Children's School,37.778864,-122.414584,"{'type': 'Point', 'coordinates': [-122.4145837..."
4,5ef9f16145de338078b9ac47,John Muir Elementary School,37.773624,-122.428363,"{'type': 'Point', 'coordinates': [-122.4283626..."


In [8]:
curse = db.basketball.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
schools = list(curse)
df_schools= pd.DataFrame(schools)
df_schools.head()

,_id,name,latitude,longitude,location
0,5ef9f13145de338078b9ac13,The Fillmore,37.783952,-122.433033,"{'type': 'Point', 'coordinates': [-122.4330327..."
1,5ef9f13145de338078b9ac14,The Armory,37.768075,-122.420768,"{'type': 'Point', 'coordinates': [-122.4207681..."
2,5ef9f13145de338078b9ac15,DNA Lounge,37.770956,-122.412689,"{'type': 'Point', 'coordinates': [-122.4126888..."
3,5ef9f13145de338078b9ac16,Chase Center,37.767864,-122.387463,"{'type': 'Point', 'coordinates': [-122.387463,..."
4,5ef9f13145de338078b9ac17,24 Hour Fitness,37.766728,-122.429692,"{'type': 'Point', 'coordinates': [-122.4296917..."


Buscaremos el punto exacto desde este punto (37.7836386,-122.398522) ya que en el mapa hemos visto que es donde podemos encontrar mayor numero de empresas de gaming, más restaurantes veganos, clubs , colegios y pistas de basketball.



In [9]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

In [12]:
def getlocationeNear(address, maxDist=500):
    point = geocode(address)
    return {
       "location": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }

In [13]:
gamer=getlocationeNear("SIE, 400 2nd Street, San Francisco, CA 94017, Estados Unidos") #la empresa de diseño de juegos que vale más de un millón más cerca según el punto
gamer

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-122.39516, 37.78431]},
   '$maxDistance': 500}}}

In [14]:
cur = db.gamings.find(gamer, {"_id":0})
print(cur.count())
result = list(cur)
result

12


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'Rupture',
  'total_money_raised': '$3M',
  'description': 'Headquarters',
  'address1': '400 2nd Street',
  'address2': 'Suite 350',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.783898,
  'longitude': -122.395234,
  'location': {'type': 'Point', 'coordinates': [-122.395234, 37.783898]}},
 {'name': 'Shattered Reality Interactive',
  'total_money_raised': '$1.1M',
  'description': 'Main Office',
  'address1': '490 2nd Street',
  'address2': 'Suite 100',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.783046,
  'longitude': -122.394067,
  'location': {'type': 'Point', 'coordinates': [-122.394067, 37.783046]}},
 {'name': 'Shattered Reality Interactive',
  'total_money_raised': '$1.1M',
  'description': 'Main Office',
  'address1': '490 2nd Street',
  'address2': 'Suite 100',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',


In [24]:
vegan=getlocationeNear("Dottie's True Blue Cafe, San Francisco, California", maxDist=600) #el restaurante vegano  más cerca según el punto

vegan

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-122.44049, 37.78806]},
   '$maxDistance': 600}}}

In [46]:
cur = db.gamings.find(vegan, {"_id":0})
print(cur.count())
result = list(cur)
result

0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[]

In [47]:
school=getlocationeNear("375 7th St, San Francisco, CA 94103, United States",maxDist=1000) #el colegio   más cerca según el punto

school

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-122.40615, 37.7754]},
   '$maxDistance': 1000}}}

In [48]:
cur = db.gamings.find(school, {"_id":0})
print(cur.count())
result = list(cur)
result

5


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'ngmoco',
  'total_money_raised': '$44.8M',
  'description': 'Secret Factory',
  'address1': '185 Berry Street',
  'address2': 'Suite 3000',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.7705096,
  'longitude': -122.4015807,
  'location': {'type': 'Point', 'coordinates': [-122.4015807, 37.7705096]}},
 {'name': 'Citizen Sports',
  'total_money_raised': '$10M',
  'description': '',
  'address1': '525 Brannan St',
  'address2': '',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.777513,
  'longitude': -122.397044,
  'location': {'type': 'Point', 'coordinates': [-122.397044, 37.777513]}},
 {'name': 'GotGame',
  'total_money_raised': '$1M',
  'description': 'GotGame HQ',
  'address1': '1 Bluxome Place',
  'address2': 'Suite 204',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.77772,


In [49]:
Club=getlocationeNear("826 Folsom St, San Francisco, CA 94107, United States", maxDist=500) #el club   más cerca según el punto

Club

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-122.40193, 37.78142]},
   '$maxDistance': 500}}}

In [50]:
cur = db.gamings.find(Club, {"_id":0})
print(cur.count())
result = list(cur)
result

2


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'IODA',
  'total_money_raised': '$0',
  'description': 'null',
  'address1': '539 Bryant Street, Suite 303',
  'address2': 'null',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.780134,
  'longitude': -122.396744,
  'location': {'type': 'Point', 'coordinates': [-122.396744, 37.780134]}},
 {'name': 'Scavenja',
  'total_money_raised': '$0',
  'description': '',
  'address1': '539 Bryant St.,',
  'address2': 'Suite 402',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.780134,
  'longitude': -122.396744,
  'location': {'type': 'Point', 'coordinates': [-122.396744, 37.780134]}}]

In [51]:
court=getlocationeNear("375 11th St, San Francisco, CA 94103, United States", maxDist=1000) #la cancha   más cerca según el punto

court

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-122.41286, 37.77093]},
   '$maxDistance': 1000}}}

In [52]:
cur = db.gamings.find(court, {"_id":0})
print(cur.count())
result = list(cur)
result

1


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'ngmoco',
  'total_money_raised': '$44.8M',
  'description': 'Secret Factory',
  'address1': '185 Berry Street',
  'address2': 'Suite 3000',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.7705096,
  'longitude': -122.4015807,
  'location': {'type': 'Point', 'coordinates': [-122.4015807, 37.7705096]}}]


Tras localizar escuelas, clubs, restaurantes veganos,courts de basketbalgamers companies que valen más de un millón ,la compañia seleccionada que aparece en 4 busquedas es  Scavenja. Por lo que procedemos a verificar a que distancia se encuentra de los requirimientos:

In [74]:
# Calculamos distance para la oficina elegida "Scavenja" que cumple con los criterios requeridos:
office_coor =(37.780134,-122.396744)
court_coor= (37.7710, -122.413)
club_coor= (37.7816,-122.402)
school_coor= (37.7763,-122.406)
vegan_coor=(37.785751395,-122.404123709)
office_near_coor = (37.7839,-122.395)
airport_coor = (37.7857,-122.410)


In [75]:
print("club",geodesic(office_coor,club_coor))
print("court",geodesic(office_coor,court_coor))
print("school",geodesic(office_coor,school_coor))
print("vegan_restaurant",geodesic(office_coor,vegan_coor))
print("offices_near",geodesic(office_coor,office_near_coor))
print("airport",geodesic(office_coor,airport_coor))



club 0.4907764314669059 km
court 1.7546611083781394 km
school 0.9197809838870833 km
vegan_restaurant 0.900747605269903 km
offices_near 0.4453363851813138 km
airport 1.3210798150082281 km


In [ ]:
##la conclusión la desarrollo a través del read me